In [1]:
# import things
import os
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from torchvision import transforms
import scipy
from PIL import Image
import gpsm
import gc
from tqdm import tqdm
from pathlib import Path
from transformers import Dinov2Config, Dinov2Model, AutoImageProcessor
PATH = '/home/anastasia/models/'

In [2]:
def cleanup(): 
    try: 
        del fmri_model
    except NameError:
        pass
    try:
        del embed_model
    except NameError:
        pass
    try:
        del image_processor
    except NameError:
        pass
    try:
        del d
    except NameError:
        pass
    try:
        del y_l
    except NameError:
        pass
    try:
        del y_r
    except NameError:
        pass
    try:
        del features
    except NameError:
        pass
    try: 
        del out_lh
    except NameError:
        pass 
    try:
        del out_rh
    except NameError:
        pass

In [3]:
device = 'cuda'

# load fmri
subj = 'subj01'
data_dir = '/home/anastasia/datasets/fmri/subjects/' + subj
fmri_dir = os.path.join(data_dir, 'training_split', 'training_fmri')
lh_fmri = np.load(os.path.join(fmri_dir, 'lh_training_fmri.npy'))
rh_fmri = np.load(os.path.join(fmri_dir, 'rh_training_fmri.npy'))

print('LH training fMRI data shape:')
print(lh_fmri.shape)
print('(Training stimulus images × LH vertices)')

print('\nRH training fMRI data shape:')
print(rh_fmri.shape)
print('(Training stimulus images × RH vertices)')

LH training fMRI data shape:
(9841, 19004)
(Training stimulus images × LH vertices)

RH training fMRI data shape:
(9841, 20544)
(Training stimulus images × RH vertices)


In [4]:
def select_roi_fmri(data_dir, fmri_full, hem, roi):
  # Define the ROI class based on the selected ROI
  if roi in ["V1v", "V1d", "V2v", "V2d", "V3v", "V3d", "hV4"]:
    roi_class = 'prf-visualrois'
  elif roi in ["EBA", "FBA-1", "FBA-2", "mTL-bodies"]:
      roi_class = 'floc-bodies'
  elif roi in ["OFA", "FFA-1", "FFA-2", "mTL-faces", "aTL-faces"]:
      roi_class = 'floc-faces'
  elif roi in ["OPA", "PPA", "RSC"]:
      roi_class = 'floc-places'
  elif roi in ["OWFA", "VWFA-1", "VWFA-2", "mfs-words", "mTL-words"]:
      roi_class = 'floc-words'
  elif roi in ["early", "midventral", "midlateral", "midparietal", "ventral", "lateral", "parietal"]:
      roi_class = 'streams'

  # Load the ROI brain surface maps
  challenge_roi_class_dir = os.path.join(data_dir, 'roi_masks',
      hem[0]+'h.'+roi_class+'_challenge_space.npy')
  roi_map_dir = os.path.join(data_dir, 'roi_masks',
      'mapping_'+roi_class+'.npy')
  challenge_roi_class = np.load(challenge_roi_class_dir)
  roi_map = np.load(roi_map_dir, allow_pickle=True).item()

  # Select the vertices corresponding to the ROI of interest
  roi_mapping = list(roi_map.keys())[list(roi_map.values()).index(roi)]
  challenge_roi = np.asarray(challenge_roi_class == roi_mapping, dtype=int)
  # Get the subset of fmri data
  idx_roi = np.where(challenge_roi)[0]
  the_fmri = fmri_full[:,idx_roi]

  mapping_size = the_fmri.shape[1] # number of vertices in this roi

  return the_fmri, mapping_size, idx_roi, challenge_roi

In [5]:
# load images 
train_img_dir  = os.path.join(data_dir, 'training_split', 'training_images')
test_img_dir  = os.path.join(data_dir, 'test_split', 'test_images')

# Create lists will all training and test image file names, sorted
train_img_list = os.listdir(train_img_dir)
train_img_list.sort()
test_img_list = os.listdir(test_img_dir)
test_img_list.sort()
print('Training images: ' + str(len(train_img_list)))
print('Test images: ' + str(len(test_img_list)))

train_img_file = train_img_list[0]
print('Training image file name: ' + train_img_file)
print('73k NSD images ID: ' + train_img_file[-9:-4])

Training images: 9841
Test images: 159
Training image file name: train-0001_nsd-00013.png
73k NSD images ID: 00013


In [6]:
# create train, validation and test partitions
rand_seed = 5 
np.random.seed(rand_seed)

# Calculate how many stimulus images correspond to 90% of the training data
num_train = int(np.round(len(train_img_list) / 100 * 90))
# Shuffle all training stimulus images
idxs = np.arange(len(train_img_list))
np.random.shuffle(idxs)
# Assign 90% of the shuffled stimulus images to the training partition,
# and 10% to the test partition
idxs_train, idxs_val = idxs[:num_train], idxs[num_train:]
# No need to shuffle or split the test stimulus images
idxs_test = np.arange(len(test_img_list))

print('Training stimulus images: ' + format(len(idxs_train)))
print('Validation stimulus images: ' + format(len(idxs_val)))
print('Test stimulus images: ' + format(len(idxs_test)))

Training stimulus images: 8857
Validation stimulus images: 984
Test stimulus images: 159


In [7]:
# define image transform
transform = transforms.Compose([
    transforms.ToTensor(), transforms.Resize(244), transforms.CenterCrop(224), transforms.Normalize([0.5], [0.5])
    ])
#transform = AutoImageProcessor.from_pretrained('facebook/dinov2-base')

In [8]:
class ImageDataset(Dataset):
    def __init__(self, imgs_paths, idxs, transform, lh_fmri, rh_fmri):
        self.imgs_paths = np.array(imgs_paths)[idxs]
        self.transform = transform
        self.lh_fmri = lh_fmri[idxs] 
        self.rh_fmri = rh_fmri[idxs]
        
    def __len__(self):
        return len(self.imgs_paths)

    def __getitem__(self, idx):
        # Load the image
        img_path = self.imgs_paths[idx]
        img = Image.open(img_path).convert('RGB')
        # Preprocess the image and send it to the chosen device ('cpu' or 'cuda')
        if self.transform:
            img = self.transform(img).to(device)
        _lh_fmri = torch.tensor(self.lh_fmri[idx]).to(device)
        _rh_fmri = torch.tensor(self.rh_fmri[idx]).to(device)
        return img, _lh_fmri, _rh_fmri

In [9]:
batch_size = 8 

# Get the paths of all image files
train_imgs_paths = sorted(list(Path(train_img_dir).iterdir()))
test_imgs_paths = sorted(list(Path(test_img_dir).iterdir()))

# The DataLoaders contain the ImageDataset class
train_imgs_dataloader = DataLoader(
      ImageDataset(train_imgs_paths, idxs_train, transform, lh_fmri, rh_fmri), 
      batch_size=batch_size
  )
val_imgs_dataloader = DataLoader(
      ImageDataset(train_imgs_paths, idxs_val, transform, lh_fmri, rh_fmri), 
      batch_size=batch_size
)

In [10]:
# load images 
train_img_dir  = os.path.join(data_dir, 'training_split', 'training_images')
test_img_dir  = os.path.join(data_dir, 'test_split', 'test_images')

# Create lists will all training and test image file names, sorted
train_img_list = os.listdir(train_img_dir)
train_img_list.sort()
test_img_list = os.listdir(test_img_dir)
test_img_list.sort()
print('Training images: ' + str(len(train_img_list)))
print('Test images: ' + str(len(test_img_list)))

train_img_file = train_img_list[0]
print('Training image file name: ' + train_img_file)
print('73k NSD images ID: ' + train_img_file[-9:-4])

Training images: 9841
Test images: 159
Training image file name: train-0001_nsd-00013.png
73k NSD images ID: 00013


In [11]:
# Load model for feature embedding
embed_model = Dinov2Model.from_pretrained("facebook/dinov2-base")

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
embed_model.to(device)
embed_model.eval()

Dinov2Model(
  (embeddings): Dinov2Embeddings(
    (patch_embeddings): Dinov2PatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): Dinov2Encoder(
    (layer): ModuleList(
      (0-11): 12 x Dinov2Layer(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attention): Dinov2Attention(
          (attention): Dinov2SelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): Dinov2SelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (layer_scale1): Dinov2LayerScale()
        (drop_path): Ide

##### Model part

Here we test the linear regression which we consider as a baseline for predictions. 

In [12]:
# regression test 
from sklearn.linear_model import LinearRegression

the_fmri, mapping_size, idx_roi, mask = select_roi_fmri(data_dir, lh_fmri, 'lh', "V1v")

ss_layers = [12]

features_train = []
y_l_train = []
y_r_train = []
for it, (d,y_l,y_r) in tqdm(enumerate(train_imgs_dataloader), total=len(train_imgs_dataloader)):
    d, y_l, y_r = d.to(device), y_l, y_r
    # pass data through DinoV2
    with torch.no_grad():
        #features = embed_model(d, output_hidden_states=True).hidden_states
        features = embed_model(d).last_hidden_state
    # select subset of layers from embed_model 
    # features = torch.cat([features[i].unsqueeze(1) for i in ss_layers], dim=1) # [8, 1, 257, 768]
    features = features.flatten(1) # [8, 257 * 768]
    # features = features[:,0,:]
    features_train += [features] 
    y_l_train += [y_l[:,idx_roi]]
    y_r_train += [y_r[:,idx_roi]]

features_train = torch.cat(features_train, dim=0)
y_l_train = torch.cat(y_l_train, dim=0)
y_r_train = torch.cat(y_r_train, dim=0)
reg_fit = LinearRegression().fit(features_train.cpu().numpy(), y_l_train.cpu().numpy())

del features_train
del y_l_train
del y_r_train
gc.collect()
torch.cuda.empty_cache()

features_val = []
y_l_val = []
y_r_val = []
for it, (d,y_l,y_r) in tqdm(enumerate(val_imgs_dataloader), total=len(val_imgs_dataloader)):
    d, y_l, y_r = d.to(device), y_l, y_r
    # pass data through DinoV2
    with torch.no_grad():
        # features = embed_model(d, output_hidden_states=True).hidden_states
        features = embed_model(d).last_hidden_state
    # select subset of layers from embed_model 
    # features = torch.cat([features[i].unsqueeze(1) for i in ss_layers], dim=1)
    features = features.flatten(1) # [8, 257 * 768]
    # features = features[:,0,:]
    features_val += [features]
    y_l_val += [y_l[:,idx_roi]]
    y_r_val += [y_r[:,idx_roi]]

features_val = torch.cat(features_val, dim=0)
y_l_val = torch.cat(y_l_val, dim=0).cpu().numpy()
y_r_val = torch.cat(y_r_val, dim=0).cpu().numpy()
val_preds = reg_fit.predict(features_val.cpu().numpy())

del features_val
gc.collect()
torch.cuda.empty_cache()

# Empty correlation array of shape: (LH vertices)
correlation = np.zeros(val_preds.shape[1])
# Correlate each predicted LH vertex with the corresponding ground truth vertex
for v in range(val_preds.shape[1]):
    correlation[v] = scipy.stats.pearsonr(val_preds[:,v],y_l_val[:,v])[0]
print('\nCorrelation is '+ str(np.mean(correlation)))

del y_l_val
del y_r_val
gc.collect()
torch.cuda.empty_cache()

100%|██████████| 123/123 [01:59<00:00,  1.03it/s]



Correlation is 0.45105930244016057
